In [0]:
pip install catboost

     |████████████████████████████████| 64.8MB 70kB/s 


In [0]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn import *
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

In [0]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive

/content/gdrive/My Drive


In [0]:
train = pd.read_excel('train_banking.xlsx')
test = pd.read_excel('test_banking.xlsx')

In [0]:
train['Interest_Rate'].value_counts()

2    70580
3    59923
1    33806
Name: Interest_Rate, dtype: int64

In [0]:
train.head(2)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,7000,< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,30000,4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3


In [0]:
test.head(2)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10164310,27500,10+ years,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male
1,10164311,26000,10+ years,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male


In [0]:
train1 = train.drop('Interest_Rate', axis = 1)

In [0]:
combined = train1.append(test)

In [0]:
combined.head(2)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10000001,7000,< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female
1,10000002,30000,4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female


In [0]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273850 entries, 0 to 109540
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_ID                  273850 non-null  int64  
 1   Loan_Amount_Requested    273850 non-null  int64  
 2   Length_Employed          261543 non-null  object 
 3   Home_Owner               231790 non-null  object 
 4   Annual_Income            231850 non-null  float64
 5   Income_Verified          273850 non-null  object 
 6   Purpose_Of_Loan          273850 non-null  object 
 7   Debt_To_Income           273850 non-null  float64
 8   Inquiries_Last_6Mo       273850 non-null  int64  
 9   Months_Since_Deliquency  126612 non-null  float64
 10  Number_Open_Accounts     273850 non-null  int64  
 11  Total_Accounts           273850 non-null  int64  
 12  Gender                   273850 non-null  object 
dtypes: float64(3), int64(5), object(5)
memory usage: 29.3+ MB


In [0]:
combined.columns

Index(['Loan_ID', 'Loan_Amount_Requested', 'Length_Employed', 'Home_Owner',
       'Annual_Income', 'Income_Verified', 'Purpose_Of_Loan', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Months_Since_Deliquency', 'Number_Open_Accounts',
       'Total_Accounts', 'Gender'],
      dtype='object')

In [0]:
for column in ['Length_Employed', 'Home_Owner',]:
    combined[column].fillna(combined[column].mode()[0], inplace=True)

In [0]:
combined['Annual_Income'].fillna(combined['Annual_Income'].mean(), inplace=True)

In [0]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273850 entries, 0 to 109540
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_ID                  273850 non-null  int64  
 1   Loan_Amount_Requested    273850 non-null  int64  
 2   Length_Employed          273850 non-null  object 
 3   Home_Owner               273850 non-null  object 
 4   Annual_Income            273850 non-null  float64
 5   Income_Verified          273850 non-null  object 
 6   Purpose_Of_Loan          273850 non-null  object 
 7   Debt_To_Income           273850 non-null  float64
 8   Inquiries_Last_6Mo       273850 non-null  int64  
 9   Months_Since_Deliquency  126612 non-null  float64
 10  Number_Open_Accounts     273850 non-null  int64  
 11  Total_Accounts           273850 non-null  int64  
 12  Gender                   273850 non-null  object 
dtypes: float64(3), int64(5), object(5)
memory usage: 29.3+ MB


In [0]:
train2 = combined.iloc[:164309]

In [0]:
test2 = combined.iloc[164309:]

In [0]:
train3 = pd.merge(train2, train[['Loan_ID','Interest_Rate']], left_on='Loan_ID',right_on='Loan_ID',how='inner')

In [0]:
train3.shape

(164309, 14)

In [0]:
test3 = test2

In [0]:
train4 = train3.drop('Months_Since_Deliquency', axis =1)
test4 = test3.drop('Months_Since_Deliquency', axis =1)

In [0]:
train4.columns

Index(['Loan_ID', 'Loan_Amount_Requested', 'Length_Employed', 'Home_Owner',
       'Annual_Income', 'Income_Verified', 'Purpose_Of_Loan', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Number_Open_Accounts', 'Total_Accounts',
       'Gender', 'Interest_Rate'],
      dtype='object')

In [0]:
train4[['Loan_ID', 'Loan_Amount_Requested',
       'Annual_Income', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Number_Open_Accounts', 'Total_Accounts',]] = train4[['Loan_ID', 'Loan_Amount_Requested',
       'Annual_Income', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Number_Open_Accounts', 'Total_Accounts',]].astype(int)

In [0]:
test4[['Loan_ID', 'Loan_Amount_Requested',
       'Annual_Income', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Number_Open_Accounts', 'Total_Accounts',]] = test4[['Loan_ID', 'Loan_Amount_Requested',
       'Annual_Income', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Number_Open_Accounts', 'Total_Accounts',]].astype(int)

In [0]:
# dummy = pd.get_dummies(train4)
# train5 = pd.concat([train4, dummy], axis = 1)

In [0]:
dummy1 = pd.get_dummies(train4)
train5 = train4.merge(dummy1, left_index=True, right_index=True)

In [0]:
dummy2 = pd.get_dummies(test4)
test5 = test4.merge(dummy2, left_index=True, right_index=True)

In [0]:
train5.head(2)

,Loan_ID_x,Loan_Amount_Requested_x,Length_Employed,Home_Owner,Annual_Income_x,Income_Verified,Purpose_Of_Loan,Debt_To_Income_x,Inquiries_Last_6Mo_x,Number_Open_Accounts_x,Total_Accounts_x,Gender,Interest_Rate_x,Loan_ID_y,Loan_Amount_Requested_y,Annual_Income_y,Debt_To_Income_y,Inquiries_Last_6Mo_y,Number_Open_Accounts_y,Total_Accounts_y,Interest_Rate_y,Length_Employed_1 year,Length_Employed_10+ years,Length_Employed_2 years,Length_Employed_3 years,Length_Employed_4 years,Length_Employed_5 years,Length_Employed_6 years,Length_Employed_7 years,Length_Employed_8 years,Length_Employed_9 years,Length_Employed_< 1 year,Home_Owner_Mortgage,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Income_Verified_VERIFIED - income,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Purpose_Of_Loan_car,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Gender_Female,Gender_Male
0,10000001,7000,< 1 year,Rent,68000,not verified,car,18,0,9,14,Female,1,10000001,7000,68000,18,0,9,14,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,10000002,30000,4 years,Mortgage,73392,VERIFIED - income,debt_consolidation,14,0,12,24,Female,3,10000002,30000,73392,14,0,12,24,3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
test5.head(2)

,Loan_ID_x,Loan_Amount_Requested_x,Length_Employed,Home_Owner,Annual_Income_x,Income_Verified,Purpose_Of_Loan,Debt_To_Income_x,Inquiries_Last_6Mo_x,Number_Open_Accounts_x,Total_Accounts_x,Gender,Loan_ID_y,Loan_Amount_Requested_y,Annual_Income_y,Debt_To_Income_y,Inquiries_Last_6Mo_y,Number_Open_Accounts_y,Total_Accounts_y,Length_Employed_1 year,Length_Employed_10+ years,Length_Employed_2 years,Length_Employed_3 years,Length_Employed_4 years,Length_Employed_5 years,Length_Employed_6 years,Length_Employed_7 years,Length_Employed_8 years,Length_Employed_9 years,Length_Employed_< 1 year,Home_Owner_Mortgage,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Income_Verified_VERIFIED - income,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Purpose_Of_Loan_car,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Gender_Female,Gender_Male
0,10164310,27500,10+ years,Mortgage,129000,VERIFIED - income,debt_consolidation,12,0,10,37,Male,10164310,27500,129000,12,0,10,37,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,10164311,26000,10+ years,Mortgage,110000,not verified,credit_card,11,0,6,23,Male,10164311,26000,110000,11,0,6,23,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
print(train5.shape)
print(test5.shape)

(164309, 56)
(109541, 54)


In [0]:
x_train = train5.drop(['Loan_ID_x', 'Loan_Amount_Requested_x', 'Length_Employed', 'Home_Owner',
       'Annual_Income_x', 'Income_Verified', 'Purpose_Of_Loan',
       'Debt_To_Income_x', 'Inquiries_Last_6Mo_x', 'Number_Open_Accounts_x',
       'Total_Accounts_x', 'Gender', 'Interest_Rate_x','Interest_Rate_y'], axis =1)
y_train = train5['Interest_Rate_y']
x_test = test5.drop(['Loan_ID_x', 'Loan_Amount_Requested_x', 'Length_Employed', 'Home_Owner',
       'Annual_Income_x', 'Income_Verified', 'Purpose_Of_Loan',
       'Debt_To_Income_x', 'Inquiries_Last_6Mo_x', 'Number_Open_Accounts_x',
       'Total_Accounts_x', 'Gender'], axis =1)

In [0]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(164309, 42)
(164309,)
(109541, 42)


In [0]:
estimator = [
             ('GBC', GradientBoostingClassifier(n_estimators=800, learning_rate=0.05, max_depth=8,verbose=1)),
             ('LGBMC' , LGBMClassifier(objective='multiclass', boosting='gbdt', learning_rate = 0.05, max_depth = 8, n_estimators = 4000,verbose=1)),
            #  ('XGBC' , XGBClassifier(max_depth=8, n_estimators=300, learning_rate=0.05))
            #  ('CBC', CatBoostClassifier(iterations=5000, learning_rate=0.01, l2_leaf_reg=3.5, depth=8, rsm=0.98, loss_function= 'MultiClass', eval_metric='AUC'))
            ]
stack_model = StackingClassifier(estimators= estimator, final_estimator= LogisticRegression(solver = 'lbfgs'))
stack_model.fit(x_train, y_train)
y_pred = stack_model.predict(x_test)
arr = pd.DataFrame(y_pred)

      Iter       Train Loss   Remaining Time 
         1      172040.9166           45.48m
         2      170688.9529           45.87m
         3      169451.7187           45.63m
         4      168311.1673           45.38m
         5      167266.0876           45.21m
         6      166296.5075           45.15m
         7      165411.0942           45.14m
         8      164581.5888           45.09m
         9      163812.2053           45.05m
        10      163098.0488           44.95m
        20      157876.9812           44.52m
        30      154729.7847           43.88m
        40      152604.0812           43.03m
        50      150939.0548           42.21m
        60      149676.5330           41.40m
        70      148663.6401           40.58m
        80      147787.2897           39.83m
        90      146947.8300           39.12m
       100      146170.2394           38.44m
       200      140268.3729           32.22m
       300      135717.9114           26.57m
       40

In [0]:
model = LGBMClassifier(objective='multiclass', boosting='gbdt', learning_rate = 0.05, max_depth = 8, n_estimators = 1000,verbose=1)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
arr = pd.DataFrame(y_pred)

In [58]:
arr

,0
0,2
1,2
2,3
3,2
4,2
...,...
109536,3
109537,2
109538,2
109539,3


In [0]:
arr[0].value_counts()

2    63495
3    32158
1    13888
Name: 0, dtype: int64

In [59]:
arr[0].value_counts()

2    62215
3    40855
1     6471
Name: 0, dtype: int64

In [0]:
from google.colab import files
arr.to_csv('arr9.csv')
files.download('arr9.csv')